We already saw that we can add attributes to instances at run-time, and that it affects just that single instance:


In [2]:

class Person:
    pass

In [3]:
p1 = Person()
p2 = Person()

p1.name = 'Alex'


In [5]:
p1.__dict__, p2.__dict__


({'name': 'Alex'}, {})

So what happens if we add a function as an attribute to our instances directly (we can even do the same within an `__init__` method, works the same way)?

Remember that if we add a function to the class itself, calling the function from the instance will result in a method.

Here, the result is different, since we are adding the function directly to the instance, not the class:

In [6]:
p1.say_hello = lambda: 'Hello!'


In [7]:
p1.__dict__

{'name': 'Alex', 'say_hello': <function __main__.<lambda>()>}

In [8]:
p1.say_hello()


'Hello!'

Of course, the other instances do not know anything about that function:

In [9]:
p2.__dict__

{}


### ## So, the question becomes, can we create a <i>method</i> on a specific instance?

The answer (of course!) is yes, but we have to explicitly tell Python we are setting up a method bound to that specific instance.

We do this by creating a method type object: <b>`from types import MethodType`</b>
1. where MethodType(function, object): function--> function we want to bind, object--> the object to bind to.

In [10]:
from types import MethodType

In [12]:
class Person:
    def __init__(self, name):
        self.name = name

In [13]:
p1 = Person('Eric')
p2 = Person('Alex')

Now let's create a method object, and bind it to p1. First we create a function that will handle the bound object as it's first argument, and use the instance name property.

In [14]:
def say_hello(self):
    return f'{self.name} says hello!'

Now we can use that function just by itself, passing in any object that has a name attribute:

In [16]:
say_hello(p1), say_hello(p2)

('Eric says hello!', 'Alex says hello!')

Now however, we are going to create a method bound to p1 specifically:

In [17]:
p1_say_hello = MethodType(say_hello, p1)

In [18]:
p1_say_hello

<bound method say_hello of <__main__.Person object at 0x0000023136FA1320>>

As you can see that method is bound to the instance p1. But how do we call it?

If we try to use dotted notation or a getattr, that won't work because the p1 object does not know anything about that method:

In [20]:
try:
    p1.p1_say_hello()
except AttributeError as ex:
    print(ex)

'Person' object has no attribute 'p1_say_hello'


All we need to do is add that method to the instance dictionary - giving it whatever name we want:

In [22]:
p1.say_hello = p1_say_hello

In [23]:
p1.__dict__


{'name': 'Eric',
 'say_hello': <bound method say_hello of <__main__.Person object at 0x0000023136FA1320>>}

OK, so now out instance knows about that method that we stored under the name say_hello:



In [24]:
p1.say_hello(), getattr(p1, 'say_hello')()

('Eric says hello!', 'Eric says hello!')

And of course, othe instances know nothing about this:



In [25]:
p2.__dict__

{'name': 'Alex'}


So, to create a bound method after the object has initially been created, we just create a bound method and add it to the instance itself.

We can do it this way (what we just saw):

In [26]:
p1 = Person('Alex')
p1.__dict__

{'name': 'Alex'}

In [27]:
p1.say_hello = MethodType(lambda self: f'{self.name} says hello', p1)

In [28]:
p1.say_hello()

'Alex says hello'

But we can also do this from any instance method too.

##### Example

Suppose we want some class to have some functionality that is called the same way but will differ from instance to instance. Although we could use inheritance, here I want some kind of 'plug-in' approach and we can do this without inheritance, mixins, or anything like that!

In [30]:
from types import MethodType

class Person:
    def __init__(self, name):
        self.name = name
        
    def register_do_work(self, func):
        setattr(self, '_do_work', MethodType(func, self))
        
    def do_work(self):
        do_work_method = getattr(self, '_do_work', None)
        # if attribute exists we'll get it back, otherwise it will be None
        if do_work_method:
            return do_work_method()
        else:
            raise AttributeError('You must first register a do_work method')

In [31]:
math_teacher = Person('Eric')
english_teacher = Person('John')

Right now neither the math nor the english teacher can do any woirk because we haven't "registered" a worker yet:



In [32]:
try:
    math_teacher.do_work()
except AttributeError as ex:
    print(ex)

You must first register a do_work method


Ok, so let's do that:

In [33]:

def work_math(self):
     return f'{self.name} will teach differentials today.'

In [34]:
math_teacher.register_do_work(work_math)


In [35]:
math_teacher.__dict__


{'name': 'Eric',
 '_do_work': <bound method work_math of <__main__.Person object at 0x0000023136FA1F60>>}

In [36]:
math_teacher.do_work()


'Eric will teach differentials today.'

And we can create a different do_work method for the English teacher:



In [38]:
def work_english(self):
    return f'{self.name} will analyze Hamlet today.'

In [39]:
english_teacher.register_do_work(work_english)


In [40]:
english_teacher.do_work()


'John will analyze Hamlet today.'